### 1. Libraries & Function


In [7]:
import os
import warnings
import logging
import pandas as pd
import matplotlib.pyplot as plt
from oemof.solph import (Bus, EnergySystem, Flow, Model, create_time_index, processing)
from oemof.solph.components import (Sink, Source, Converter, GenericStorage)
from oemof.solph import EnergySystem
from oemof.solph import views
import oemof.solph as solph
import pandas as pd

def LCOH(invest_cost, operation_cost, heat_produced, revenue=0, i=0.05, n=20):
    pvf = ((1 + i) ** n - 1) / ((1 + i) ** n * i)
    return (invest_cost + pvf * (operation_cost - revenue)) / (
        pvf * heat_produced
    )
def epc(invest_cost, i=0.05, n=20):
    af = (i * (1 + i) ** n) / ((1 + i) ** n - 1)
    return invest_cost * af

filename = r"OEMOF_files/input_data.csv"
data = pd.read_csv(filename)
solver = "cbc"
solver_verbose = False

print(data.columns.tolist())

['thermal_demand_MW', 'electricity_price_Euro_Per_MWh', 'co2_price', 'gas_price_Euro_Per_MWh', 'biomass_Euro_Per_MWh']


### 2. Energy System

In [8]:
datetimeindex = create_time_index(2022, number=len(data)) 
energysystem = EnergySystem(timeindex=datetimeindex, infer_last_interval=False)

electrical_bus = Bus(label="electrical_bus") # Electricity bus
thermal_bus = Bus(label="thermal_bus") # Heat bus
gas_bus = Bus(label="gas_bus") # Natural gas bus
biomass_bus = Bus(label="biomass_bus") # Biomass bus
solar_bus = Bus(label="solar_bus") # Solar bus
energysystem.add(electrical_bus, thermal_bus, gas_bus, biomass_bus, solar_bus)

# print(len(energysystem.timeindex))
# print(data["gas_price_Euro_Per_MWh"].shape)
# print(data["gas_price_Euro_Per_MWh"].dtype)


energysystem.add(
    Source(label="natural_gas",outputs={gas_bus: Flow(variable_costs=data["gas_price_Euro_Per_MWh"])})
                )
energysystem.add(
    Source(label="electricity_grid",outputs={electrical_bus: Flow(variable_costs=data["electricity_price_Euro_Per_MWh"])})
                )
energysystem.add(
    Source(label="biomass",outputs={electrical_bus: Flow(variable_costs=data["biomass_Euro_Per_MWh"])})
                )

# Excess electricity sink
energysystem.add(
    Sink(
        label="excess_electricity",
        inputs={electrical_bus: Flow(variable_costs=data["electricity_price_Euro_Per_MWh"] * -1)}
    )
)

energysystem.add(
    Sink(
        label="thermal_demand_MW",
        inputs={thermal_bus: Flow(
            nominal_value=data["thermal_demand_MW"].max(),
            fix=data["thermal_demand_MW"]
        )},
    )
)
Natural_Gas_CHP_cap = 3 # MW
energysystem.add(
    Converter(
        label="Natural_Gas_CHP",
        inputs={gas_bus: Flow()},
        outputs={thermal_bus: Flow(nominal_value=Natural_Gas_CHP_cap*0.81, variable_costs=1),
                 electrical_bus: Flow(nominal_value=Natural_Gas_CHP_cap*0.16, variable_costs=1)},
        conversion_factors={thermal_bus: 0.51, 
                            electrical_bus: 0.46}
    )
)
Biomass_CHP_cap = 5  # MW
energysystem.add(
    Converter(
        label="Biomass_CHP",
        inputs={biomass_bus: Flow()},
        outputs={thermal_bus: Flow(nominal_value=Biomass_CHP_cap*0.81, variable_costs=1),
                 electrical_bus: Flow(nominal_value=Biomass_CHP_cap*0.16, variable_costs=1)},
        conversion_factors={thermal_bus: 0.81, 
                            electrical_bus: 0.16}
    )
)



model = Model(energysystem)


logging.info("Solving the optimization problem.")
model.solve(
    solver='cbc',
    solve_kwargs={"tee": True},
    cmdline_options={"ratioGap": "0.02"}
)

# Process results
energysystem.results["main"] = processing.results(model)
# energysystem.results["meta"] = processing.meta_results(model)

# Save results to file
output_file = os.path.join(os.getcwd(), "Outputs/results.oemof")

energysystem.dump(os.getcwd(), "Outputs/results.oemof")

logging.info("Results have been dumped.")

Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Jul 31 2025 

command line - C:\Users\ardia\miniconda3\envs\env_P2\Library\bin\cbc.exe -ratioGap 0.02 -printingOptions all -import C:\Users\ardia\AppData\Local\Temp\tmp1c6rzqbj.pyomo.lp -stat=1 -solve -solu C:\Users\ardia\AppData\Local\Temp\tmp1c6rzqbj.pyomo.soln (default strategy 1)
ratioGap was changed from 0 to 0.02
Option for printingOptions changed from normal to all
Presolve determined that the problem was infeasible with tolerance of 1e-08
Presolved model looks infeasible - will use unpresolved


Problem has 70080 rows, 87600 columns (69892 with objective) and 157680 elements
There are 34852 singletons with objective 188 singletons with no objective 
Column breakdown:
52560 of type 0.0->inf, 35040 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
61320 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
87

  - termination condition: infeasible
  - message from solver: <undefined>
c:\Users\ardia\miniconda3\envs\env_P2\lib\site-packages\oemof\solph\_models.py:263: UserWarning: Optimization ended with status warning and termination condition infeasible
  warnings.warn(
c:\Users\ardia\miniconda3\envs\env_P2\lib\site-packages\oemof\network\energy_system.py:256: FutureWarning: Parameter 'dpath' will be removed in a future version. You can give the directory as part of the filename and set 'consider_dpath' to False to suppress this waring.
  warnings.warn(
